In [ ]:
pip install newspaper3k

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.1/211.1 KB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.9/93.9 KB 4.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.1/81.1 KB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 23.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for tinysegmenter: filename=tinysegmenter-0.3-py3-none-any.whl size=13552 sha256=23611dcc722b3e38600614d859cb54f4af5986bc12f9364a6b4ff2608e2f6647
  Stored in directory: /root/.cache/pip/wheels/99/74/83/8fac1c8d9c648cfabebbbffe97a889f6624817f3aa0bbe6c09
  Created wheel for feedfinder2: filename=feedfinder2-0.0.4-py3-none-any.whl size=3354 sha256=f84bb52b2179a6caf0cb31f2ab85ce77e891beaac3be875c064b8e

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pwd
import os
os.chdir('/content/drive/MyDrive/Colab Notebooks/Hybrid-Based-Text-Characterisation-for-Biased-Statement-Identification-in-News-Articles')
!pwd

/content/drive/My Drive/Colab Notebooks/Hybrid-Based-Text-Characterisation-for-Biased-Statement-Identification-in-News-Articles
/content/drive/MyDrive/Colab Notebooks/Hybrid-Based-Text-Characterisation-for-Biased-Statement-Identification-in-News-Articles


In [ ]:
import pandas as pd
import numpy as np

import nltk
nltk.download('punkt')

from newspaper import Article
import newspaper

from functools import reduce

from sklearn.model_selection import train_test_split

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
def autoScrapNSummarize(urlList, size):
  c = 0
  f = 0
  
  news_summary_scrap = []
  for i in urlList:
    if c < size:
      target_article = Article(i, language="en")
      try:
        target_article.download()
        target_article.parse()
        target_article.nlp()
        if target_article.summary != "":
          news_summary_scrap.append(target_article.summary)
          c = c + 1
        else:
          print('***FAILED TO GENERATE SUMMARY***', target_article.url)
          news_summary_scrap.append("")
          f = f + 1
      except:
        print('***FAILED TO DOWNLOAD***', target_article.url)
        news_summary_scrap.append("")
        f = f + 1
        continue
    else:
      news_summary_scrap.append("")
  print('***TOTAL DOWNLOAD COUNT***', c)
  print('***TOTAL FAILED TO DOWNLOAD***', f)
  return news_summary_scrap

In [ ]:
df_nonpartisan = pd.read_csv("Datasets/news_headlines_usa_neutral.csv")
df_nonpartisan.head()

,stories_id,publish_date,title,url,language,ap_syndicated,themes,media_id,media_name,media_url
0,663692044,2020-07-12 08:00:00,"""Don't Tase Me, Bro"" tops '07 memorable quote ...",http://www.reuters.com/article/us-quotes-odd-i...,en,False,NaN,4442,Reuters,http://www.reuters.com
1,668844035,2020-07-11 08:00:00,U.S. healthcare falls short in survey of 7 nat...,http://www.reuters.com/article/us-healthcare-s...,en,False,NaN,4442,Reuters,http://www.reuters.com
2,669761491,2020-07-10 08:00:00,Nicotine may ease Parkinson's symptoms: U.S. s...,https://www.reuters.com/article/us-parkinsons-...,en,False,NaN,4442,Reuters,http://www.reuters.com
3,680460617,2020-11-05 07:00:00,Experts Divided Over Safety of Indian Point Nu...,http://www.reuters.com/article/idUS38169798202...,en,False,NaN,4442,Reuters,http://www.reuters.com
4,803694205,2020-06-15 00:00:00,"Reports: Petraeus off the list, Trump down to ...",http://thehill.com/blogs/blog-briefing-room/ne...,en,False,NaN,18364,Hill,http://thehill.com/rss/syndicator/19109


In [ ]:
df_nonpartisan['media_name'].value_counts()

Reuters                                 22616
Yahoo News - Latest News & Headlines    16072
CNBC                                    12229
USA Today                               12086
AP                                      11052
Bloomberg                                8445
Hill                                      643
Name: media_name, dtype: int64

In [ ]:
df_nonpartisan_filtered = df_nonpartisan.loc[df_nonpartisan['media_name'].isin(['Reuters', 'CNBC', 'Hill'])]
df_nonpartisan_filtered['media_name'].value_counts()

Reuters    22616
CNBC       12229
Hill         643
Name: media_name, dtype: int64

In [ ]:
df_slight_partisan_filtered = df_nonpartisan.loc[df_nonpartisan['media_name'].isin(['Yahoo News - Latest News & Headlines', 'USA Today', 'AP'])]
df_slight_partisan_filtered['media_name'].value_counts()

Yahoo News - Latest News & Headlines    16072
USA Today                               12086
AP                                      11052
Name: media_name, dtype: int64

In [ ]:
df_nonpartisan_filtered['slant'] = 0
df_nonpartisan_filtered['nonpartisan'] = 1
df_nonpartisan_filtered['slight-partisan'] = 0
df_nonpartisan_filtered['hyperpartisan'] = 0
df_nonpartisan_filtered = df_nonpartisan_filtered.drop(['stories_id', 'publish_date', 'language', 'ap_syndicated', 'themes', 'media_id', 'media_name', 'media_url'], axis=1)
df_nonpartisan_filtered.head()

<ipython-input-10-db8bd0bc62ef>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_nonpartisan_filtered['slant'] = 0
<ipython-input-10-db8bd0bc62ef>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_nonpartisan_filtered['nonpartisan'] = 1
<ipython-input-10-db8bd0bc62ef>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/use

,title,url,slant,nonpartisan,slight-partisan,hyperpartisan
0,"""Don't Tase Me, Bro"" tops '07 memorable quote ...",http://www.reuters.com/article/us-quotes-odd-i...,0,1,0,0
1,U.S. healthcare falls short in survey of 7 nat...,http://www.reuters.com/article/us-healthcare-s...,0,1,0,0
2,Nicotine may ease Parkinson's symptoms: U.S. s...,https://www.reuters.com/article/us-parkinsons-...,0,1,0,0
3,Experts Divided Over Safety of Indian Point Nu...,http://www.reuters.com/article/idUS38169798202...,0,1,0,0
4,"Reports: Petraeus off the list, Trump down to ...",http://thehill.com/blogs/blog-briefing-room/ne...,0,1,0,0


In [ ]:
df_nonpartisan_filtered.shape

(35488, 6)

In [ ]:
df_nonpartisan_filtered_rus = df_nonpartisan_filtered.sample(frac=0.20)

In [ ]:
df_nonpartisan_filtered_rus.shape

(7098, 6)

In [ ]:
df_nonpartisan_filtered_rus.head()

,title,url,slant,nonpartisan,slight-partisan,hyperpartisan
6071,Bloomberg qualifies for next Democratic debate,https://www.cnbc.com/2020/02/18/bloomberg-qual...,0,1,0,0
58998,"Norwegian CEO sees lower 2021 bookings, hopes ...",https://www.cnbc.com/2020/09/22/norwegian-ceo-...,0,1,0,0
19774,Fed's Neel Kashkari says coronavirus economic ...,https://www.cnbc.com/2020/04/12/coronavirus-fe...,0,1,0,0
9210,California holds cruise ship offshore for coro...,http://feeds.reuters.com/~r/Reuters/domesticNe...,0,1,0,0
62351,U.S. Senate panel to take up FAA aircraft cert...,https://www.reuters.com/article/us-boeing-737m...,0,1,0,0


In [ ]:
df_nonpartisan_filtered_rus.dropna(subset=['url'], inplace=True)

In [ ]:
df_nonpartisan_filtered_rus.shape

(7098, 6)

In [ ]:
df_url_nonpartisan = df_nonpartisan_filtered_rus.loc[:, ['url']]
url_nonpartisan_list = df_url_nonpartisan['url'].values.tolist()

In [ ]:
df_url_nonpartisan.head()

,url
6071,https://www.cnbc.com/2020/02/18/bloomberg-qual...
58998,https://www.cnbc.com/2020/09/22/norwegian-ceo-...
19774,https://www.cnbc.com/2020/04/12/coronavirus-fe...
9210,http://feeds.reuters.com/~r/Reuters/domesticNe...
62351,https://www.reuters.com/article/us-boeing-737m...


In [ ]:
news_summary_scrap_nonpartisan = autoScrapNSummarize(urlList = url_nonpartisan_list, size = 3500)

***FAILED TO DOWNLOAD*** http://feeds.reuters.com/~r/Reuters/domesticNews/~3/-BBc4wY1hFY/california-holds-cruise-ship-offshore-for-coronavirus-screening-idUSKBN20S0QE
***FAILED TO DOWNLOAD*** http://feeds.reuters.com/~r/reuters/companyNews/~3/0iKcYApbViU/morning-news-call-india-may-6-idUSL4N2CO0JS
***FAILED TO DOWNLOAD*** http://feeds.reuters.com/~r/reuters/USVideoLatest/~3/qG2Ul1GFNoA/trump-blames-cdc-obama-for-coronavirus-t
***FAILED TO DOWNLOAD*** http://feeds.reuters.com/~r/reuters/companyNews/~3/oLII1PDHlVk/update-2-australia-dives-into-qe-cuts-rates-as-virus-roils-economy-idUSL4N2BC1EE
***FAILED TO DOWNLOAD*** http://feeds.reuters.com/~r/Reuters/worldNews/~3/s4AUPwTveGo/even-as-canada-reopens-children-in-foster-care-endure-long-separations-idUSKBN2321EI
***FAILED TO DOWNLOAD*** http://feeds.reuters.com/~r/reuters/companyNews/~3/a4TRuf7gS40/update-2-uk-in-talks-with-roche-on-game-changer-covid-19-antibody-tests-idUSL4N2CV3SA
***FAILED TO DOWNLOAD*** http://feeds.reuters.com/~r/reu

In [ ]:
news_summary_scrap_nonpartisan

['Democratic presidential candidate former New York City Mayor Mike Bloomberg delivers remarks during a campaign rally on February 12, 2020 in Nashville, Tennessee.\nThey have until 11:59 p.m. on Feb. 18 to qualify for the debate, which is being hosted by NBC News, MSNBC and the Nevada Independent.\nFormer New York City Mayor Mike Bloomberg on Tuesday qualified for his first appearance in a Democratic presidential primary debate.\nBloomberg spokeswoman Galia Slayen told NBC News on Monday that he would participate in the debate if he qualified.\n"If you look at the new DNC debate thresholds as anything other than, \'Get Andrew Yang off stage and put Mike Bloomberg on,\' then you are high," the adviser, Brad Bauman, tweeted after the announcement.',
 'Cruise bookings for 2021 are down, CEO of Norwegian Cruise Lines Frank Del Rio told CNBC on Tuesday, acknowledging the sustained impact the coronavirus pandemic is having on global travel and the industry.\nDel Rio said future bookings are

In [ ]:
d = {'url': url_nonpartisan_list, 'news_summary': news_summary_scrap_nonpartisan}
df_news_summary_scrap_nonpartisan = pd.DataFrame(data=d)

In [ ]:
df_news_summary_scrap_nonpartisan.head()

,url,news_summary
0,https://www.cnbc.com/2020/02/18/bloomberg-qual...,Democratic presidential candidate former New Y...
1,https://www.cnbc.com/2020/09/22/norwegian-ceo-...,"Cruise bookings for 2021 are down, CEO of Norw..."
2,https://www.cnbc.com/2020/04/12/coronavirus-fe...,Minneapolis Federal Reserve President Neel Kas...
3,http://feeds.reuters.com/~r/Reuters/domesticNe...,
4,https://www.reuters.com/article/us-boeing-737m...,(Reuters) - The U.S. Senate Commerce Committee...


In [ ]:
merge_list = [df_nonpartisan_filtered_rus, df_news_summary_scrap_nonpartisan]
df_nonpartisan_filtered_rus_final = reduce(lambda df_nonpartisan_filtered_rus, df_news_summary_scrap_nonpartisan: pd.merge(df_nonpartisan_filtered_rus , df_news_summary_scrap_nonpartisan, on = ["url"], how = "inner"), merge_list)

In [ ]:
df_nonpartisan_filtered_rus_final = df_nonpartisan_filtered_rus_final[df_nonpartisan_filtered_rus_final.news_summary != ""]

In [ ]:
df_nonpartisan_filtered_rus_final

,title,url,slant,nonpartisan,slight-partisan,hyperpartisan,news_summary
0,Bloomberg qualifies for next Democratic debate,https://www.cnbc.com/2020/02/18/bloomberg-qual...,0,1,0,0,Democratic presidential candidate former New Y...
1,"Norwegian CEO sees lower 2021 bookings, hopes ...",https://www.cnbc.com/2020/09/22/norwegian-ceo-...,0,1,0,0,"Cruise bookings for 2021 are down, CEO of Norw..."
2,Fed's Neel Kashkari says coronavirus economic ...,https://www.cnbc.com/2020/04/12/coronavirus-fe...,0,1,0,0,Minneapolis Federal Reserve President Neel Kas...
4,U.S. Senate panel to take up FAA aircraft cert...,https://www.reuters.com/article/us-boeing-737m...,0,1,0,0,(Reuters) - The U.S. Senate Commerce Committee...
6,Coronavirus live updates: U.S. nears 4 million...,https://www.cnbc.com/2020/07/23/coronavirus-li...,0,1,0,0,Members of the Infectious Disease Society of A...
...,...,...,...,...,...,...,...
4945,"US STOCKS-Wall Street ends higher, led by ener...",https://www.reuters.com/article/usa-stocks/us-...,0,1,0,0,"The S&P 500 energy, materials industrial , hea..."
4946,NBA players protest racial injustice as league...,https://www.reuters.com/article/us-basketball-...,0,1,0,0,(Reuters) - NBA teams bowed their heads and kn...
4947,"Fired over text, 800 calls to unemployment: Wh...",https://www.cnbc.com/2020/04/22/what-its-like-...,0,1,0,0,The U.S. is currently grappling with unprecede...
4949,How a biotech CEO figured out Covid-19 surveil...,https://www.cnbc.com/2020/08/14/this-biotech-c...,0,1,0,0,The only option to identify these asymptomatic...


In [ ]:
df_slight_partisan_filtered['slant'] = 1
df_slight_partisan_filtered['nonpartisan'] = 0
df_slight_partisan_filtered['slight-partisan'] = 1
df_slight_partisan_filtered['hyperpartisan'] = 0
df_slight_partisan_filtered = df_slight_partisan_filtered.drop(['stories_id', 'publish_date', 'language', 'ap_syndicated', 'themes', 'media_id', 'media_name', 'media_url'], axis=1)
df_slight_partisan_filtered.head()

<ipython-input-31-a0b49f3371c8>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_slight_partisan_filtered['slant'] = 1
<ipython-input-31-a0b49f3371c8>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_slight_partisan_filtered['nonpartisan'] = 0
<ipython-input-31-a0b49f3371c8>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/st

,title,url,slant,nonpartisan,slight-partisan,hyperpartisan
11,"2020 resolution for Trump, Democrats and us: L...",http://rssfeeds.usatoday.com/~/614316404/0/usa...,1,0,1,0
12,"New year, new laws, new (?) decade: 5 things y...",http://rssfeeds.usatoday.com/~/614324396/0/usa...,1,0,1,0
21,Protesters at US Embassy in Baghdad disperse a...,http://rssfeeds.usatoday.com/~/614379142/0/usa...,1,0,1,0
26,Chief Justice John Roberts: 'We have come to t...,http://rssfeeds.usatoday.com/~/614408496/0/usa...,1,0,1,0
27,Giuliani says he would be willing to testify i...,http://rssfeeds.usatoday.com/~/614398776/0/usa...,1,0,1,0


In [ ]:
df_slight_partisan_filtered.shape

(39210, 6)

In [ ]:
df_slight_partisan_filtered_rus = df_slight_partisan_filtered.sample(frac=0.20)

In [ ]:
df_slight_partisan_filtered_rus.shape

(7842, 6)

In [ ]:
df_slight_partisan_filtered_rus.head()

,title,url,slant,nonpartisan,slight-partisan,hyperpartisan
78101,Second dose of COVID vaccine from Pfizer or Mo...,https://news.yahoo.com/second-dose-covid-vacci...,1,0,1,0
47370,'We're very far apart': Coronavirus relief neg...,http://rssfeeds.usatoday.com/~/632720220/0/usa...,1,0,1,0
68711,Citizen Science Effort Launched for Gaming Com...,https://apnews.com/PR%20Newswire/32a77d4535dc4...,1,0,1,0
44616,Kansas Senate race tests GOP leaders' power to...,https://news.yahoo.com/kansas-senate-race-test...,1,0,1,0
78033,"As Biden Defense secretary, Austin would shatt...",http://rssfeeds.usatoday.com/~/639997972/0/usa...,1,0,1,0


In [ ]:
df_slight_partisan_filtered_rus.dropna(subset=['url'], inplace=True)

In [ ]:
df_slight_partisan_filtered_rus.shape

(7842, 6)

In [ ]:
df_url_slight_partisan = df_slight_partisan_filtered_rus.loc[:, ['url']]
url_slight_partisan_list = df_url_slight_partisan['url'].values.tolist()

In [ ]:
df_url_slight_partisan.head()

,url
78101,https://news.yahoo.com/second-dose-covid-vacci...
47370,http://rssfeeds.usatoday.com/~/632720220/0/usa...
68711,https://apnews.com/PR%20Newswire/32a77d4535dc4...
44616,https://news.yahoo.com/kansas-senate-race-test...
78033,http://rssfeeds.usatoday.com/~/639997972/0/usa...


In [ ]:
news_summary_scrap_slight_partisan = autoScrapNSummarize(urlList = url_slight_partisan_list, size = 3500)

***FAILED TO DOWNLOAD*** https://news.yahoo.com/5-weapons-made-nazi-germany-131500248.html
***FAILED TO DOWNLOAD*** https://news.yahoo.com/china-heavy-steps-hong-kong-170500185.html
***FAILED TO DOWNLOAD*** https://news.yahoo.com/indelible-stain-gop-tried-topple-165957371.html
***FAILED TO DOWNLOAD*** https://news.yahoo.com/monday-sunrise-briefing-record-early-061916888.html
***FAILED TO DOWNLOAD*** https://news.yahoo.com/americas-m2-carbine-rifle-made-163000838.html
***FAILED TO DOWNLOAD*** https://news.yahoo.com/america-bombed-nazi-germany-back-230000394.html
***FAILED TO DOWNLOAD*** https://news.yahoo.com/why-even-atlanta-way-faces-141356716.html
***FAILED TO DOWNLOAD*** https://news.yahoo.com/el-salvador-bar-attack-six-113650988.html
***FAILED TO DOWNLOAD*** https://news.yahoo.com/americans-catching-virus-increasingly-no-162255058.html
***FAILED TO DOWNLOAD*** https://news.yahoo.com/trump-biden-debate-two-old-223302818.html
***FAILED TO DOWNLOAD*** https://news.yahoo.com/flynns-exo

In [ ]:
news_summary_scrap_slight_partisan

['But during a pandemic, when the risk of infection is high, he noted that people are better off getting the second shot – and being fully protected – according to the authorized schedule.\n"If there is significant transmission of disease, as is the case here, we should absolutely get the second dose exactly as has been studied," he said.\nBut the level of protection increases dramatically – from 52% to 95% – after the second shot, he said, and traditionally second shots provide longer-lasting immunity than a single dose.\nEveryone in the study got a second shot roughly 21 days after the first, so it\'s unclear whether protection from a single dose would be long-lasting.\nThis article originally appeared on USA TODAY: COVID vaccine: Timing of second Pfizer and Moderna shots not critical',
 'WASHINGTON – Negotiations over another coronavirus relief package tumbled into a state of uncertainty Thursday evening with congressional leaders and lead White House negotiators painting a grim pic

In [ ]:
d = {'url': url_slight_partisan_list, 'news_summary': news_summary_scrap_slight_partisan}
df_news_summary_scrap_slight_partisan = pd.DataFrame(data=d)

In [ ]:
df_news_summary_scrap_slight_partisan.head()

,url,news_summary
0,https://news.yahoo.com/second-dose-covid-vacci...,"But during a pandemic, when the risk of infect..."
1,http://rssfeeds.usatoday.com/~/632720220/0/usa...,WASHINGTON – Negotiations over another coronav...
2,https://apnews.com/PR%20Newswire/32a77d4535dc4...,"DALLAS, April 27, 2020 /PRNewswire/ -- BALANCE..."
3,https://news.yahoo.com/kansas-senate-race-test...,"Kobach, the former Kansas secretary of state, ..."
4,http://rssfeeds.usatoday.com/~/639997972/0/usa...,"As of May 2020, the total number of West Point..."


In [ ]:
merge_list = [df_slight_partisan_filtered_rus, df_news_summary_scrap_slight_partisan]
df_slight_partisan_filtered_rus_final = reduce(lambda df_slight_partisan_filtered_rus, df_news_summary_scrap_slight_partisan: pd.merge(df_slight_partisan_filtered_rus , df_news_summary_scrap_slight_partisan, on = ["url"], how = "inner"), merge_list)

In [ ]:
df_slight_partisan_filtered_rus_final = df_slight_partisan_filtered_rus_final[df_slight_partisan_filtered_rus_final.news_summary != ""]

In [ ]:
df_slight_partisan_filtered_rus_final

,title,url,slant,nonpartisan,slight-partisan,hyperpartisan,news_summary
0,Second dose of COVID vaccine from Pfizer or Mo...,https://news.yahoo.com/second-dose-covid-vacci...,1,0,1,0,"But during a pandemic, when the risk of infect..."
1,'We're very far apart': Coronavirus relief neg...,http://rssfeeds.usatoday.com/~/632720220/0/usa...,1,0,1,0,WASHINGTON – Negotiations over another coronav...
2,Citizen Science Effort Launched for Gaming Com...,https://apnews.com/PR%20Newswire/32a77d4535dc4...,1,0,1,0,"DALLAS, April 27, 2020 /PRNewswire/ -- BALANCE..."
3,Kansas Senate race tests GOP leaders' power to...,https://news.yahoo.com/kansas-senate-race-test...,1,0,1,0,"Kobach, the former Kansas secretary of state, ..."
4,"As Biden Defense secretary, Austin would shatt...",http://rssfeeds.usatoday.com/~/639997972/0/usa...,1,0,1,0,"As of May 2020, the total number of West Point..."
...,...,...,...,...,...,...,...
3582,Redesigned Ford F-150 pickup 'on track' despit...,http://rssfeeds.usatoday.com/~/624063660/0/usa...,1,0,1,0,The redesigned Ford F-150 remains scheduled to...
3583,Ginseng piles up in Canada during pandemic des...,https://news.yahoo.com/ginseng-piles-canada-du...,1,0,1,0,Canada is the world's second-largest ginseng e...
3584,Murkowski wants to hear case before deciding o...,https://news.yahoo.com/murkowski-wants-hear-ca...,1,0,1,0,The Republican said Saturday she wants to make...
3585,Texas Dems Declare ‘Big War’ on Gov Over Mail-...,https://news.yahoo.com/texas-dems-declare-big-...,1,0,1,0,“We’re in a big war right now with the state o...


In [ ]:
df_partisan = pd.read_csv("Datasets/news_headlines_usa_biased.csv")
df_partisan.head()

,stories_id,publish_date,title,url,language,ap_syndicated,themes,media_id,media_name,media_url
0,900954563,2020-04-15 00:00:00,One Day U: Going Back to College for One Day |...,https://www.huffingtonpost.com/renee-fisher/on...,en,False,NaN,27502,HuffPost,http://www.huffingtonpost.com/#
1,802518660,2020-11-15 00:00:00,Watch Donald Trump Compassionately Defend Drea...,https://www.huffingtonpost.com/entry/trump-def...,en,False,NaN,27502,HuffPost,http://www.huffingtonpost.com/#
2,869218430,2020-04-18 00:00:00,'Whitewashing War Crimes': How UK Academics Pr...,https://www.huffingtonpost.co.uk/entry/uk-acad...,en,False,NaN,27502,HuffPost,http://www.huffingtonpost.com/#
3,1001112150,2020-06-18 00:00:00,Mexican Official Says Girl With Down Syndrome ...,https://www.huffingtonpost.co.uk/entry/mexican...,en,False,NaN,27502,HuffPost,http://www.huffingtonpost.com/#
4,1483890044,NaN,"Todd Bensman, Author at The Federalist",https://thefederalist.com/author/toddbensman/,en,False,NaN,366282,Federalist,http://thefederalist.com/


In [ ]:
df_partisan['media_name'].value_counts()

Breitbart             11276
HuffPost               7467
Daily Caller           6687
Alternet               4885
Daily Beast            4611
Daily Wire             3170
Slate                  2502
American Greatness     1948
New Yorker             1617
Federalist             1442
Name: media_name, dtype: int64

In [ ]:
df_partisan_filtered = df_partisan.loc[df_partisan['media_name'].isin(['Breitbart', 'HuffPost', 'Daily Caller', 'Alternet', 'Daily Beast', 'Daily Wire'])]
df_partisan_filtered['media_name'].value_counts()

Breitbart       11276
HuffPost         7467
Daily Caller     6687
Alternet         4885
Daily Beast      4611
Daily Wire       3170
Name: media_name, dtype: int64

In [ ]:
df_partisan_filtered['slant'] = 2
df_partisan_filtered['nonpartisan'] = 0
df_partisan_filtered['slight-partisan'] = 0
df_partisan_filtered['hyperpartisan'] = 1
df_partisan_filtered = df_partisan_filtered.drop(['stories_id', 'publish_date', 'language', 'ap_syndicated', 'themes', 'media_id', 'media_name', 'media_url'], axis=1)
df_partisan_filtered.head()

<ipython-input-50-0e0298a7bcb5>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_partisan_filtered['slant'] = 2
<ipython-input-50-0e0298a7bcb5>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_partisan_filtered['nonpartisan'] = 0
<ipython-input-50-0e0298a7bcb5>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guid

,title,url,slant,nonpartisan,slight-partisan,hyperpartisan
0,One Day U: Going Back to College for One Day |...,https://www.huffingtonpost.com/renee-fisher/on...,2,0,0,1
1,Watch Donald Trump Compassionately Defend Drea...,https://www.huffingtonpost.com/entry/trump-def...,2,0,0,1
2,'Whitewashing War Crimes': How UK Academics Pr...,https://www.huffingtonpost.co.uk/entry/uk-acad...,2,0,0,1
3,Mexican Official Says Girl With Down Syndrome ...,https://www.huffingtonpost.co.uk/entry/mexican...,2,0,0,1
5,Netflix’s Weird Show About a ‘Messiah’ From th...,http://feedproxy.google.com/~r/thedailybeast/a...,2,0,0,1


In [ ]:
df_partisan_filtered.shape

(38096, 6)

In [ ]:
df_partisan_filtered_rus = df_partisan_filtered.sample(frac=0.20)

In [ ]:
df_partisan_filtered_rus.shape

(7619, 6)

In [ ]:
df_url_partisan_filtered_rus = df_partisan_filtered_rus.loc[:, ['url']]
url_partisan_list = df_url_partisan_filtered_rus['url'].values.tolist()

In [ ]:
df_url_partisan_filtered_rus.shape

(7619, 1)

In [ ]:
df_url_partisan_filtered_rus.head()

,url
32310,https://www.huffpost.com/entry/stephen-colbert...
8033,http://feedproxy.google.com/~r/breitbart/~3/wI...
37834,https://www.thedailybeast.com/even-when-hes-co...
15090,https://www.dailywire.com/news/final-numbers-s...
25552,https://www.thedailybeast.com/these-trump-camp...


In [ ]:
news_summary_scrap_partisan = autoScrapNSummarize(urlList = url_partisan_list, size = 3500)

***FAILED TO DOWNLOAD*** http://feedproxy.google.com/~r/breitbart/~3/wIT4R0udmlo/
***FAILED TO DOWNLOAD*** http://feedproxy.google.com/~r/breitbart/~3/JH230XVMqXI/
***FAILED TO DOWNLOAD*** http://feedproxy.google.com/~r/breitbart/~3/IUcAV7uiRZA/
***FAILED TO DOWNLOAD*** http://feedproxy.google.com/~r/thedailybeast/articles/~3/-Mh_wVfS7zQ/border-militia-leader-larry-hopkins-pleads-guilty-to-gun-charge-his-followers-say-its-fake-news
***FAILED TO DOWNLOAD*** http://feedproxy.google.com/~r/breitbart/~3/7x_qA54b9vE/
***FAILED TO DOWNLOAD*** http://feedproxy.google.com/~r/breitbart/~3/0CyFRbhPdVk/
***FAILED TO DOWNLOAD*** http://feedproxy.google.com/~r/breitbart/~3/C-bqd89Awpc/
***FAILED TO DOWNLOAD*** http://feedproxy.google.com/~r/breitbart/~3/vqGzoUXAero/
***FAILED TO DOWNLOAD*** http://feedproxy.google.com/~r/breitbart/~3/6h-dCFt-u8A/
***FAILED TO DOWNLOAD*** http://feedproxy.google.com/~r/breitbart/~3/uDS8xlVFVno/
***FAILED TO DOWNLOAD*** http://feedproxy.google.com/~r/breitbart/~3/rM0

In [ ]:
news_summary_scrap_partisan

['This is one “uncle” that Stephen Colbert won’t be inviting to Sunday dinner.\n“The Late Show” host on Wednesday branded Donald Trump as “America’s racist uncle” after The Washington Post reported on the president’s bigotry out of the public eye.\n“President Trump has maintained that Black Americans have mainly themselves to blame in their struggle for equality, hindered more by lack of initiative than societal impediments,” Colbert said, quoting a passage in the article.\n“Why the hell did I do that?” he yelled at aides, a former official said.\nHe also reportedly muttered that Jews “are only in it for themselves.”Quipped Colbert: “Donald Trump saying you’re only in it for yourself is like Jeffrey Dahmer telling you to be a vegan.”',
 '',
 'On the other hand, I got a teeny rush when I remembered this was the last debate and that there may soon be no more President Trump to watch.\nTrump pivoted to Biden’s 1994 Crime Bill and the many Black folks it condemned to jail, as if he expecte

In [ ]:
d = {'url': url_partisan_list, 'news_summary': news_summary_scrap_partisan}
df_news_summary_scrap_partisan = pd.DataFrame(data=d)

In [ ]:
df_news_summary_scrap_partisan.head()

,url,news_summary
0,https://www.huffpost.com/entry/stephen-colbert...,This is one “uncle” that Stephen Colbert won’t...
1,http://feedproxy.google.com/~r/breitbart/~3/wI...,
2,https://www.thedailybeast.com/even-when-hes-co...,"On the other hand, I got a teeny rush when I r..."
3,https://www.dailywire.com/news/final-numbers-s...,"Bloomberg announced on November 24, 2019 that ..."
4,https://www.thedailybeast.com/these-trump-camp...,The Trump campaign’s payments to the Trump Org...


In [ ]:
merge_list = [df_partisan_filtered_rus, df_news_summary_scrap_partisan]
df_partisan_filtered_rus_final = reduce(lambda df_partisan_filtered_rus, df_news_summary_scrap_partisan: pd.merge(df_partisan_filtered_rus, df_news_summary_scrap_partisan, on = ["url"], how = "outer"), merge_list)

In [ ]:
df_partisan_filtered_rus_final = df_partisan_filtered_rus_final[df_partisan_filtered_rus_final.news_summary != ""]

In [ ]:
df_partisan_filtered_rus_final

,title,url,slant,nonpartisan,slight-partisan,hyperpartisan,news_summary
0,Colbert Calls Trump ‘America’s Racist Uncle’ A...,https://www.huffpost.com/entry/stephen-colbert...,2,0,0,1,This is one “uncle” that Stephen Colbert won’t...
2,"Even When He’s Courting Us, Trump Keeps Insult...",https://www.thedailybeast.com/even-when-hes-co...,2,0,0,1,"On the other hand, I got a teeny rush when I r..."
3,Final Numbers Show Michael Bloomberg Actually ...,https://www.dailywire.com/news/final-numbers-s...,2,0,0,1,"Bloomberg announced on November 24, 2019 that ..."
4,These Trump Campaign Payments to the Trump Org...,https://www.thedailybeast.com/these-trump-camp...,2,0,0,1,The Trump campaign’s payments to the Trump Org...
5,Trump's Response When Asked If He Regrets All ...,https://www.huffpost.com/entry/trump-asked-reg...,2,0,0,1,"Donald Trump, a president whose persistent unt..."
...,...,...,...,...,...,...,...
4833,"In Puerto Rico, Conservatives Sweep Island-Wid...",https://www.huffpost.com/entry/puerto-rico-ele...,2,0,0,1,A man carries a Puerto Rican flag during a pro...
4836,Biden Campaign Video Features Ukrainian Activi...,https://dailycaller.com/2020/01/21/biden-video...,2,0,0,1,Former Vice President Joe Biden’s campaign rel...
4837,Moderna Vaccine Shows Success In Early Clinica...,https://www.dailywire.com/news/moderna-vaccine...,2,0,0,1,Biotech company Moderna reported early signs o...
4839,"After Trump’s Long Night, Kamala Ushers In a J...",https://www.thedailybeast.com/joy-comes-in-the...,2,0,0,1,Aunt Gerry lived a life with few aspirations a...


In [ ]:
frames = [df_nonpartisan_filtered_rus_final, df_slight_partisan_filtered_rus_final, df_partisan_filtered_rus_final]
df = pd.concat(frames, ignore_index = True)
df

,title,url,slant,nonpartisan,slight-partisan,hyperpartisan,news_summary
0,Bloomberg qualifies for next Democratic debate,https://www.cnbc.com/2020/02/18/bloomberg-qual...,0,1,0,0,Democratic presidential candidate former New Y...
1,"Norwegian CEO sees lower 2021 bookings, hopes ...",https://www.cnbc.com/2020/09/22/norwegian-ceo-...,0,1,0,0,"Cruise bookings for 2021 are down, CEO of Norw..."
2,Fed's Neel Kashkari says coronavirus economic ...,https://www.cnbc.com/2020/04/12/coronavirus-fe...,0,1,0,0,Minneapolis Federal Reserve President Neel Kas...
3,U.S. Senate panel to take up FAA aircraft cert...,https://www.reuters.com/article/us-boeing-737m...,0,1,0,0,(Reuters) - The U.S. Senate Commerce Committee...
4,Coronavirus live updates: U.S. nears 4 million...,https://www.cnbc.com/2020/07/23/coronavirus-li...,0,1,0,0,Members of the Infectious Disease Society of A...
...,...,...,...,...,...,...,...
10495,"In Puerto Rico, Conservatives Sweep Island-Wid...",https://www.huffpost.com/entry/puerto-rico-ele...,2,0,0,1,A man carries a Puerto Rican flag during a pro...
10496,Biden Campaign Video Features Ukrainian Activi...,https://dailycaller.com/2020/01/21/biden-video...,2,0,0,1,Former Vice President Joe Biden’s campaign rel...
10497,Moderna Vaccine Shows Success In Early Clinica...,https://www.dailywire.com/news/moderna-vaccine...,2,0,0,1,Biotech company Moderna reported early signs o...
10498,"After Trump’s Long Night, Kamala Ushers In a J...",https://www.thedailybeast.com/joy-comes-in-the...,2,0,0,1,Aunt Gerry lived a life with few aspirations a...


In [ ]:
df['slant'].value_counts()

0    3500
1    3500
2    3500
Name: slant, dtype: int64

In [ ]:
df.to_csv('Datasets/news_headlines_with_scrap_news_summary.csv', index = False)